In [ ]:
import numpy as np
import time
import pickle
import json

import math
from random import random, randint, uniform

import os
import shutil

from statistics import median

In [ ]:
def generate_falstar_problem(observations, theta, v, radius=10): 
    
    f_str = "(define-system F\n"
    f_str += "\t(simulink \"flight_2018.slx\")\n"
    f_str += "\t(parameters simTime v init_theta)\n"
    f_str += "\t(inputs  w)\n"
    f_str += "\t(outputs x y)\n"
    f_str += "\t(piecewise-constant w -0.31415927 0.31415927)\n"
    f_str += "\t(constant simTime 100)\n"
    f_str += "\t(constant v {})\n".format(v)
    f_str += "\t(constant init_theta {})\n".format(theta)
    f_str += "\t)\n\n"
    
    f_str += "(define radius {})\n\n".format(radius)
    
    for i in range(len(observations)):
        obs = observations[i]
        f_str += "(define cx{} {})\n".format(i+1, obs[0])
        f_str += "(define cy{} {})\n\n".format(i+1, obs[1])
        
    f_str += "(define observation (cx cy)\n"
    f_str += "\t(< (+ (* (- x cx) (- x cx)) (* (- y cy) (- y cy))) (* radius radius)))\n\n"
    
    f_str += "(define seq\n"
    f_str += "\t(and {}))\n\n".format(" ".join(["(eventually (0 {}) (observation cx{} cy{}))".format(i+0.4, i, i) for i in range(1,len(observations)+1)]))
    
    
    f_str += "(set-seed   23)\n"
    f_str += "(set-repeat 10)\n"
    f_str += "(define N 1000)\n\n"

    f_str += "(select-system F)\n"
    f_str += "(set-log \"benchmarks/flight/falstar/flight.csv\")\n"
    f_str += "(set-report \"benchmarks/flight/falstar/summary.csv\")\n\n"

    num_control_points = int(len(observations)/2*5 + 1)
    f_str += "(set-solver adaptive ({} {}) 0.25 N)\n".format(num_control_points, num_control_points)

    f_str += "(falsify (not seq))"
    
    return f_str

In [ ]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
clean_start = True

dfolder = "benchmarks/flight/falstar"
if clean_start:
    if os.path.exists(dfolder):
        shutil.rmtree(dfolder)
    os.makedirs(dfolder)

with open("flight.pkl", "rb") as f:
    instances = pickle.load(f)
    
for h in horizons:
    
    for pnum in range(num_instances):
        
        pname = 'flight_{pnum}_{h}.cfg'.format(pnum=pnum, h=h)      

        problem_str = generate_falstar_problem(instances[pnum]["observations"][:h], instances[pnum]["init_theta"], instances[pnum]["init_v"])
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(problem_str)

In [ ]:
for h in horizons:
    
    for pnum in range(num_instances):
        
        pname = 'flight_{pnum}_{h}.cfg'.format(pnum=pnum, h=h)
        cmd = "falstar " + dfolder + '/' + pname
        os.system(cmd)